# 1. Import Libraries and Set Directory
---

Import Libraries

In [10]:
is_drive = 0

In [11]:
if is_drive==1 :
    from google.colab import drive
    drive.mount('/content/drive')

In [12]:
# directory management
import os
from glob import glob
# regular imports
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
# audio analysis
import librosa 
import librosa.display
import IPython.display as ipd
# imports for colors and plots
from itertools import cycle
sns.set_theme(style="white", palette=None)
color_pal = plt.rcParams["axes.prop_cycle"].by_key()["color"]
color_cycle = cycle(plt.rcParams["axes.prop_cycle"].by_key()["color"])
# imports for NN
import tensorflow as tf
from tensorflow import keras
import time

# 3. Load Dataset and Model
---

In [20]:
# Spectogram Path
if is_drive==1 :   
    TRAIN_DIR = "/content/drive/MyDrive/spectogram-dataset/Train/"
    TEST_DIR = "/content/drive/MyDrive/spectogram-dataset/Test/"
    SPECTROGRAMS_SAVE_DIR = "/content/drive/MyDrive/autoencoder-generations/Test/"
else :
    TRAIN_DIR = "../Project/spectogram-dataset/Train/"
    TEST_DIR = "../Project/spectogram-dataset/Test/"
    SPECTROGRAMS_SAVE_DIR = "../Project/autoencoder-generations/Test/"

In [21]:
def load_fsdd(train_spectrograms_path,test_spectrograms_path):
    x_train = []
    for root, _, file_names in os.walk(train_spectrograms_path):
        for file_name in file_names:
            train_file_path = os.path.join(root, file_name)
            spectrogram = np.load(train_file_path) # (n_bins, n_frames)
            x_train.append(spectrogram)
    x_train = np.array(x_train)        # -> (n_samples, n_bins, n_frames)
    x_train = x_train[..., np.newaxis] # -> (1200, 800, 32, 1)

    x_test  = []
    for root, _, file_names in os.walk(test_spectrograms_path):
        for file_name in file_names:
            test_file_path = os.path.join(root, file_name)
            spectrogram = np.load(test_file_path) # (n_bins, n_frames)
            x_test.append(spectrogram)
    x_test = np.array(x_test)        # -> (n_samples, n_bins, n_frames)
    x_test = x_test[..., np.newaxis] # -> (300, 800, 32, 1)

    return x_train,x_test,train_file_path,test_file_path

In [22]:
_, x_test, _, test_file_path = load_fsdd(TRAIN_DIR, TEST_DIR)

In [ ]:
from autoencoder import Autoencoder
from preprocess import Saver

In [ ]:
saver = Saver(SPECTROGRAMS_SAVE_DIR)
AE = Autoencoder.load("AE-model")

In [ ]:
generated_spectrograms, latent_representations = AE.reconstruct(x_test)

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=5)
for i in range(4):
    img = librosa.display.specshow(generated_spectrograms[i],
                                x_axis='time',
                                y_axis='log',
                                ax=ax[0, i])
    ax.set_title('latent_representation', fontsize=20)
    fig.colorbar(img, ax=ax, format=f'%0.2f')
    img = librosa.display.specshow(latent_representations[i],
                                x_axis='time',
                                y_axis='log',
                                ax=ax[1, i])
    ax.set_title('generated_spectrogram', fontsize=20)
    fig.colorbar(img, ax=ax, format=f'%0.2f')
plt.show()

In [ ]:
saver.save_feature(generated_spectrograms, test_file_path)